In [1]:
import sys
sys.path.insert(1, '../')

from river import stream,tree,metrics
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque, Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

import warnings

warnings.filterwarnings("ignore")
import xgboost as xgb

import datetime, time
import utils
import sliding_window
import psutil

import copy

c:\python3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import matplotlib.animation as animation

In [3]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [4]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [5]:
'''
Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
and obtain feature matrix to transform future events
'''

# Experiment settings
dataset_label = 'bpic17'
monitoring = 'updating'

# Invoke parameters for dataset
with open('../dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']


for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    training_window_dict = dict()
    test_window_dict = dict()

    performance_measure_type = 'ROCAUC'

    dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('../result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'xgb'
    if classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01, verbosity =0)
    elif classifier == 'rf':
        streaming_classifier = RandomForestClassifier(n_estimators = 100)

    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    casecount = 0
    rowcounter = 0
    running_case = 0
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    
    # Sliding window for training setting
    window_size = 100
    test_size = 30
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier, 0]
    
    dataset = stream.iter_csv(
            dataset_loc
            )
    totallength = len(list(dataset))


    dataset = stream.iter_csv(
                dataset_loc,
                target='outcome'
                )
    enctype = 'Index-base'
    
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0

    for x,y in dataset:
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(maximum_prefix)]=set()

                x = streaming_db[caseid][maximum_prefix-1]
                training_windows.update_window(x)
 
                update_test_cases(streaming_db[caseid], test_cases)

    
                if training_windows.retraining_count == test_size:
                    x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                    for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                    feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values
                    y_training = [i.true_label for i in training_windows.container]
#                         training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training)
                    training_models['prefix_%s'%(maximum_prefix)][1] +=1
                    training_windows.reset_retraining_count()
#                         print([i.caseid for i in training_windows.container])

                    '''
                    Evaluate current model with test cases
                    '''
                    model_update_count = training_models['prefix_%s'%(maximum_prefix)][1]
                    prediction_result[maximum_prefix][model_update_count] = {}
                    y_truelist = []
                    y_predlist = []

                    training_window_dict[casecount] = copy.deepcopy(training_windows.container)
                    test_window_dict[casecount] = copy.deepcopy([c[0] for c in test_cases if len(c) >= maximum_prefix ])

#             for case in test_cases:
#                 if len(case) >= maximum_prefix:
#                     x = case[maximum_prefix-1]
#                                 model = training_models['prefix_%s'%(x.prefix_length)][0]
#                                 current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
#                                 current_event = pd.Series(current_event).to_frame().T
#                                 y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba(current_event)
#                                 y_truelist.append(x.true_label)
#                                 y_predlist.append(y_pred)

#                         prediction_result[maximum_prefix][model_update_count]['y_true'] = y_truelist
#                         prediction_result[maximum_prefix][model_update_count]['y_pred'] = y_predlist
#             if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
#                 cdhappend[maximum_prefix] = model_update_count

    print(casecount)
    if 0 not in training_window_dict.keys():
        training_window_dict[0] = {'False': 0,'True':0}
    if 0 not in test_window_dict.keys():
        test_window_dict[0] = {'False': 0,'True':0}
    label_dist_dict = {}
    label_dist_dict[0] = {'False': 0,'True':0}
    for t in range(1, casecount+1):
        if t not in training_window_dict.keys():
            label_dist_dict[t] = label_dist_dict[t-1]
        else:            
            window = training_window_dict[t]
            label_dist_dict[t] = {'False': 0,'True':0}
            for i in range(len(window)):
                if window[i].true_label == 'False':
                    label_dist_dict[t]['False'] +=1
                else:
                    label_dist_dict[t]['True'] +=1


    dist_df = pd.DataFrame.from_dict(label_dist_dict).T
    dist_df = dist_df.reset_index(drop=False)
    dist_df = dist_df.rename(columns={'index':'Case count'})
    dist_df.to_csv('../result/bpic17/prefix_%s_%s_training_window_label.csv'%(maximum_prefix, monitoring), index=False)
    
    label_dist_dict = {}
    label_dist_dict[0] = {'False': 0,'True':0}

    for t in range(1, casecount+1):
        if t not in test_window_dict.keys():
            label_dist_dict[t] = label_dist_dict[t-1]
            
        else:
            window = test_window_dict[t]
            label_dist_dict[t] = {'False': 0,'True':0}
            for i in range(len(window)):
                if window[i].true_label == 'False':
                    label_dist_dict[t]['False'] +=1
                else:
                    label_dist_dict[t]['True'] +=1            
    dist_df = pd.DataFrame.from_dict(label_dist_dict).T
    dist_df = dist_df.reset_index(drop=False)
    dist_df = dist_df.rename(columns={'index':'Case count'})
    dist_df.to_csv('../result/bpic17/prefix_%s_%s_test_window_label.csv'%(maximum_prefix, monitoring), index=False)


{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
1
1878
2
1878
3
1878
4
1878
5
1878
6
1878
7
1878
8
1878
9
1878
10
1878
11
1878
12
1878
13
1878
14
1878
15
1878


In [21]:
training_windows.container[3].caseid
train = [i.caseid for i in training_windows.container]
test = [c[0].caseid for c in test_cases]

count = 0
for t in test:
    if t in train:
        print(t)
        count+=1
print(count)

Application_1037167788
Application_1712966902
Application_394826272
Application_2097767622
Application_39469630
Application_50364786
Application_208564412
Application_1708263885
Application_1014284368
Application_1885112588
Application_1266086453
Application_292006777
Application_500032780
Application_648638072
Application_829976155
Application_1961240923
Application_1002664914
Application_236278083
Application_1474761909
Application_2002095565
Application_447427583
Application_200571610
Application_373770977
Application_1624828578
Application_1204483638
Application_443577191
Application_1357597381
Application_1968129303
Application_1104955032
Application_1988724493
30


In [19]:
label_dist_dict[0]

KeyError: 0

In [12]:
label_dist_dict

{0: {'False': 0, 'True': 0}, 1: {'False': 0, 'True': 0}}

In [21]:
training_result = '../result/bpic17/prefix_1_training_window_label.csv'
training_result =  pd.read_csv(training_result)
training_result

testing_result = '../result/bpic17/prefix_1_test_window_label.csv'
testing_result = pd.read_csv(testing_result)